# Air Quality Data per US States (cleaning)
This Python code is used to clean and prepare the air quality data for the United States only. The original data is saved in multiple CSV file. Each file containes daily data with different measurements of polluants for each country. Because of the diversity of polluants and the inconsistancy in the number of datapoints per country, only the polluant PM25 (Particule Matter with a diameter of 2.5 micrometers or less, also called PM2.5) is considered. The yearly dose is calculated as the mean over the whole year of daily medians of PM25. The results are saved in a CSV file containing the following columns: US state, year (format: YYYY) and yearly mean of PM25. 

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
from pprint import pprint

## Load data from CSV files
- Load only the data for PM25
- Load only the data for the US
- Calculate the mean of the median per day for the whole year

In [2]:
# Look only at PM 25 as a measure of air quality
aq_metrics = 'pm25'

# Get all the file extensions
exts = ['2015H1',
'2016H1',
'2017H1',
'2018H1',
'2019Q1',
'2019Q2',
'2019Q3',
'2019Q4',
'2020Q1',
'2020Q2',
'2020Q3',
'2020Q4',
'2021Q1',
'2021Q2',
'2021Q3',
'2021Q4']

# Load the first file and prepare the DataFrame
ext = exts[0]

# [1] Load CSV file to DataFrame
csvfile = Path(f"Data_Sources/AirQuality/waqi-covid-{ext}.csv")
data_01 = pd.read_csv(csvfile)

# [2] Filter for Specie = aq_metrics (should be PM25), get columns Date, Coutry, City and median
reduced_df = data_01.loc[(data_01['Specie'] == aq_metrics),['Date', 'Country', 'City', 'median']]

# [3] Group by countries and city, and calculate the mean of the medians
reduced_df = reduced_df.groupby(['Country','City']).mean('median')

# [4] Rename column median to 'PM25'
reduced_df = reduced_df.rename(columns={'median': 'PM25'})

# [5] Use the year from the CSV file name to create a column with the year
reduced_df['Year'] = ext[0:4]

# [6] Rearrange the column Year and PM25
reduced_df = reduced_df[['Year', 'PM25']]

reduced_df

Year        PM25
Country City                             
AE      Dubai            2015  118.714286
AT      Linz             2015   53.005376
        Vienna           2015   52.629630
AU      Brisbane         2015   24.947090
        Darwin           2015   26.154286
...                       ...         ...
US      The Bronx        2015   34.031746
        Tucson           2015   25.719577
        Washington D.C.  2015   37.661376
VN      Huế              2015   78.166667
        Hạ Long          2015  145.000000

[380 rows x 2 columns]

In [3]:
# Add a new CSV file to the data frame
# Perform the same operations as above [1-6], in the same order
for ext in exts[1:]:
    # Print current filename extension
    print(f"File: {ext}")

    # Operations [1-6]
    csvfile = Path(f"Data_Sources/AirQuality/waqi-covid-{ext}.csv")
    data_01 = pd.read_csv(csvfile)
    new_df = data_01.loc[(data_01['Specie'] == aq_metrics),['Date', 'Country', 'City', 'median']]
    new_df = new_df.groupby(['Country','City']).mean('median')
    new_df = new_df.rename(columns={'median': 'PM25'})
    new_df['Year'] = ext[0:4]
    new_df = new_df[['Year', 'PM25']]

    # [7] Add the DataFrame created from the CSV file to the existing DataFrame
    reduced_df = pd.concat([reduced_df, new_df])

# Once all the CSV files have been looked at, print a completion message and display the DataFrame head
print('Completed. Displaying DataFrame:')
reduced_df

File: 2016H1
File: 2017H1
File: 2018H1
File: 2019Q1
File: 2019Q2
File: 2019Q3
File: 2019Q4
File: 2020Q1
File: 2020Q2
File: 2020Q3
File: 2020Q4
File: 2021Q1
File: 2021Q2
File: 2021Q3
File: 2021Q4
Completed. Displaying DataFrame:


Year        PM25
Country City                            
AE      Dubai           2015  118.714286
AT      Linz            2015   53.005376
        Vienna          2015   52.629630
AU      Brisbane        2015   24.947090
        Darwin          2015   26.154286
...                      ...         ...
ZA      Port Elizabeth  2021   31.040816
        Pretoria        2021   51.428571
        Richards Bay    2021   17.846154
        Vereeniging     2021   42.428571
        Worcester       2021   23.021739

[7912 rows x 2 columns]

In [4]:
# Save US-only data to a new DataFrame
us_airquality_df = reduced_df.loc['US']

In [5]:
# List all the US cities
us_cities = us_airquality_df.index.tolist()
pprint(us_cities)

['Albuquerque',
 'Atlanta',
 'Austin',
 'Baltimore',
 'Boise',
 'Boston',
 'Brooklyn',
 'Charlotte',
 'Chicago',
 'Columbia',
 'Columbus',
 'Dallas',
 'Denver',
 'Detroit',
 'El Paso',
 'Fort Worth',
 'Fresno',
 'Hartford',
 'Honolulu',
 'Houston',
 'Indianapolis',
 'Jackson',
 'Jacksonville',
 'Las Vegas',
 'Little Rock',
 'Los Angeles',
 'Madison',
 'Manhattan',
 'Memphis',
 'Miami',
 'Milwaukee',
 'Nashville',
 'Oakland',
 'Oklahoma City',
 'Omaha',
 'Philadelphia',
 'Phoenix',
 'Portland',
 'Providence',
 'Queens',
 'Raleigh',
 'Richmond',
 'Sacramento',
 'Saint Paul',
 'Salem',
 'Salt Lake City',
 'San Antonio',
 'San Diego',
 'San Francisco',
 'San Jose',
 'Seattle',
 'Springfield',
 'Staten Island',
 'Tallahassee',
 'The Bronx',
 'Tucson',
 'Washington D.C.',
 'Albuquerque',
 'Atlanta',
 'Austin',
 'Baltimore',
 'Boise',
 'Boston',
 'Brooklyn',
 'Charlotte',
 'Chicago',
 'Columbia',
 'Columbus',
 'Dallas',
 'Denver',
 'Detroit',
 'El Paso',
 'Fort Worth',
 'Fresno',
 'Hartford',

In [6]:
from countries import get_state

# Iterate through the DataFrame
for index, row in us_airquality_df.iterrows():
    # Add the state based on the city
    us_airquality_df.loc[index,'State'] = get_state(index)

us_airquality_df = us_airquality_df[['State','Year','PM25']]
us_airquality_df

C:\Users\User\AppData\Local\Temp\ipykernel_524\1426095199.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airquality_df.loc[index,'State'] = get_state(index)


,State,Year,PM25
City,,,
Albuquerque,New Mexico,2015,24.054545
Atlanta,Georgia,2015,41.853261
Austin,Minnesota,2015,32.465608
Baltimore,Maryland,2015,42.973262
Boise,Idaho,2015,32.529101
...,...,...,...
Staten Island,New York,2021,32.540816
Tallahassee,Florida,2021,34.418367
The Bronx,New York,2021,29.246575


In [7]:
states_airquality_df = us_airquality_df.groupby(['State','Year']).mean('PM25')

# Save to CSV file
states_airquality_df.to_csv("Cleaned_Datasets/cleaned_airquality_usstates.csv",index=True)